In [2]:
import os


os.environ["CARAVELS_CONFIG_FILE"] = "/home/rdpds/Documents/caravels/configfiles/H01.json"

In [3]:
from datetime import datetime

from pymongo import MongoClient, DESCENDING

from utils import utils


class BuildingRepository:
    def __init__(self):
        self.config = utils.get_config()
        self.server = str(self.config['storage']['local']['server'])
        self.port = str(self.config['storage']['local']['port'])
        self.IOTS_READING = self.config['storage']['local']['iots_reading']
        self.FORECAST = self.config['storage']['local']['forecast']
        self.TOTALPOWER = self.config['storage']['local']['totalpower']
        self.TOTALPOWERHOUR = self.config['storage']['local']['totalpowerhour']
        self.CONFIG_DB = self.config['storage']['local']['config']

    def save_config(self):
        try:
            conf = {"config": "config", "auto_answer": True}
            client = MongoClient(self.server + ':' + self.port)
            client[self.CONFIG_DB[0]][self.CONFIG_DB[1]].insert_one(conf)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nConfig\n', conf)

    def get_historic_interval_iots(self, start, end):
        client = MongoClient(self.server + ':' + self.port)
        historic = list(
            client[self.IOTS_READING[0]][self.IOTS_READING[1]].find({'datetime': {'$gt': start, '$lt': end}}))
        client.close()
        return historic

    def get_historic_iots(self, start):
        if type(start) is str:
            date_start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
        else:
            date_start = start
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.IOTS_READING[0]][self.IOTS_READING[1]].find(
            {'datetime': {'$gt': date_start}}))
        client.close()
        return historic

    def get_historic_hour(self, start):
        if type(start) is str:
            date_start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
        else:
            date_start = start
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.TOTALPOWERHOUR[0]][self.TOTALPOWERHOUR[1]].find({'datetime': {'$gt': date_start}}))
        client.close()
        return historic
    
    def get_historic_hour_interval(self, start: datetime, end: datetime):
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.TOTALPOWERHOUR[0]][self.TOTALPOWERHOUR[1]].find({'datetime': {'$gt': start, '$lt': end}}))
        client.close()
        return historic

    def iots_historic_update(self, start):
        client = MongoClient(self.server + ':' + self.port)
        historic = list(client[self.IOTS_READING[0]][self.IOTS_READING[1]].find({'datetime': {'$gt': start}}))
        client.close()
        client = MongoClient(self.server + ':' + self.port)
        for entry in historic:
            new_date = datetime.strptime(entry['datetime'], "%Y-%m-%d %H:%M:%S.%f")
            client[self.IOTS_READING[0]][self.IOTS_READING[1]].update_one({'datetime': entry['datetime']},
                                                                          {'$set': {'datetime': new_date}})

        client.close()
        return historic

    def get_iots_reading_col(self, time, time_emb):
        client = MongoClient(self.server + ':' + self.port)
        building_iot_reading = list(
            client[self.IOTS_READING[0]][self.IOTS_READING[1]].find({'datetime': {'$gt': time, '$lt': time_emb}}))
        client.close()
        return building_iot_reading

    def get_forecastvalue_col(self):
        client = MongoClient(self.server + ':' + self.port)
        forecastvalue = list(client.ForecastDay.forecastvalue.find().sort("_id", DESCENDING).limit(1))
        client.close()
        return forecastvalue

    def get_power_historic(self, start):
        client = MongoClient(self.server + ':' + self.port)
        building_totalpower = list(client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].find({'datetime': {'$gt': start}}))
        client.close()
        return building_totalpower

    def get_power_historic_interval(self, start: datetime, end: datetime):
        client = MongoClient(self.server + ':' + self.port)
        building_totalpower = list(
            client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].find({'datetime': {'$gt': start, '$lt': end}}))
        client.close()
        return building_totalpower

    def get_totalpower_col(self):
        client = MongoClient(self.server + ':' + self.port)
        building_totalpower = list(
            client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].find().sort("datetime", -1).limit(100000))
        client.close()
        return building_totalpower

    def insert_total(self, totalpower, totalgeneration, datetime):
        try:
            total = {"totalpower": totalpower, "totalgeneration": totalgeneration, "datetime": datetime}
            client = MongoClient(self.server + ':' + self.port)
            client[self.TOTALPOWER[0]][self.TOTALPOWER[1]].insert_one(total)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nTotal\n', total)

    def insert_forecast(self, forecast_power, datetime):
        try:
            forecast = {"forecast_power": forecast_power, "datetime": datetime}
            client = MongoClient(self.server + ':' + self.port)
            client[self.FORECAST[0]][self.FORECAST[1]].insert_one(forecast)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nForecast\n', forecast)

    def insert_forecastday(self, iat, datetime):
        try:
            forecastday = {"forecast_power": {"0": str(iat[0, 0]),
                                              "1": str(iat[1, 0]),
                                              "2": str(iat[2, 0]),
                                              "3": str(iat[3, 0]),
                                              "4": str(iat[4, 0]),
                                              "5": str(iat[5, 0]),
                                              "6": str(iat[6, 0]),
                                              "7": str(iat[7, 0]),
                                              "8": str(iat[8, 0]),
                                              "9": str(iat[9, 0]),
                                              "10": str(iat[10, 0]),
                                              "11": str(iat[11, 0]),
                                              "12": str(iat[12, 0]),
                                              "13": str(iat[13, 0]),
                                              "14": str(iat[14, 0]),
                                              "15": str(iat[15, 0]),
                                              "16": str(iat[16, 0]),
                                              "17": str(iat[17, 0]),
                                              "18": str(iat[18, 0]),
                                              "19": str(iat[19, 0]),
                                              "20": str(iat[20, 0]),
                                              "21": str(iat[21, 0]),
                                              "22": str(iat[22, 0]),
                                              "23": str(iat[23, 0])},
                           "datetime": datetime}
            client = MongoClient(self.server + ':' + self.port)
            client.ForecastDay.forecastvalue.insert_one(forecastday)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nForecastDay\n', forecastday)

    def insert_hour(self, start_hour, consumption, generation, flexibility):
        try:
            total = {"datetime": start_hour, "consumption": consumption, "generation": generation,
                     "flexibility": flexibility}
            client = MongoClient(self.server + ':' + self.port)
            client[self.TOTALPOWERHOUR[0]][self.TOTALPOWERHOUR[1]].insert_one(total)
        except Exception as e:
            print("An exception occurred ::", e)
        finally:
            client.close()

        if self.config['app']['monitoring']:
            print('\nTotal\n', total)


In [4]:
building_repo = BuildingRepository()

In [5]:
import pandas as pd
from datetime import datetime, timedelta

In [6]:
def get_historic_last_day_by_hour():
    print(datetime.strftime(datetime.now() - timedelta(hours=24), "%Y-%m-%d %H:%M:%S"))
    total = building_repo.get_historic_hour(
        datetime.strftime(datetime.now() - timedelta(hours=24), "%Y-%m-%d %H:%M:%S"))
    if len(total)==0:
        return []
    total = pd.DataFrame(total)
    total = total.drop(["_id"], axis=1)
    total = total.values.tolist()
    return total
get_historic_last_day_by_hour()

2024-12-09 15:16:21


[[Timestamp('2024-12-10 00:00:00'),
  1586.1307371349096,
  0.0,
  153.12730180806676],
 [Timestamp('2024-12-09 23:00:00'), 1061.7041666666667, 0.0, 99.133375],
 [Timestamp('2024-12-09 22:00:00'), 1576.32545201669, 0.0, 145.95025034770515],
 [Timestamp('2024-12-09 21:00:00'),
  1540.5944444444444,
  0.0,
  146.7137361111111],
 [Timestamp('2024-12-09 20:00:00'),
  1230.7621696801114,
  0.0,
  115.80082058414463],
 [Timestamp('2024-12-09 19:00:00'), 834.6050069541029, 0.0, 76.54421418636996],
 [Timestamp('2024-12-09 18:00:00'), 771.7680555555555, 0.0, 73.47481944444445],
 [Timestamp('2024-12-09 17:00:00'), 900.6314325452016, 0.0, 86.0006675938804],
 [Timestamp('2024-12-09 16:00:00'),
  2475.223611111111,
  0.2375,
  232.50169444444444]]

In [7]:
def get_historic(start):
    total = building_repo.get_power_historic(start)
    if len(total)==0:
        return []
    total = pd.DataFrame(total)
    total = total.drop(["_id"], axis=1)
    return total.values.tolist()
get_historic(datetime.now().replace(minute=0, second=0, microsecond=0))

[[273, 382, Timestamp('2024-12-10 15:06:41.044000')]]

In [8]:
import random
def get_historic_overview():
    historic = get_historic_last_day_by_hour()
    last_hour = get_historic(datetime.now().replace(minute=0, second=0, microsecond=0))
    total = pd.DataFrame(last_hour, columns=['consumption', 'generation', 'datetime'])
    total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
    total.set_index("datetime", inplace=True)
    total = total.resample('1h').mean()
    total = total.values.tolist()[0]
    historic.insert(0, [datetime.timestamp(datetime.now().replace(minute=0, second=0, microsecond=0)), total[0], total[1],
                        total[0] * random.randrange(5, 20) / 100])
    return historic
get_historic_overview()

2024-12-09 15:16:22


[[1733842800.0, 273.0, 382.0, 40.95],
 [Timestamp('2024-12-10 00:00:00'),
  1586.1307371349096,
  0.0,
  153.12730180806676],
 [Timestamp('2024-12-09 23:00:00'), 1061.7041666666667, 0.0, 99.133375],
 [Timestamp('2024-12-09 22:00:00'), 1576.32545201669, 0.0, 145.95025034770515],
 [Timestamp('2024-12-09 21:00:00'),
  1540.5944444444444,
  0.0,
  146.7137361111111],
 [Timestamp('2024-12-09 20:00:00'),
  1230.7621696801114,
  0.0,
  115.80082058414463],
 [Timestamp('2024-12-09 19:00:00'), 834.6050069541029, 0.0, 76.54421418636996],
 [Timestamp('2024-12-09 18:00:00'), 771.7680555555555, 0.0, 73.47481944444445],
 [Timestamp('2024-12-09 17:00:00'), 900.6314325452016, 0.0, 86.0006675938804],
 [Timestamp('2024-12-09 16:00:00'),
  2475.223611111111,
  0.2375,
  232.50169444444444]]

In [9]:
def forecast_consumption():
    building_repo = BuildingRepository()
    now = datetime.now()
    start = now - timedelta(days=7, hours=now.hour, minutes=now.minute)
    building_totalpower = building_repo.get_historic_hour_interval(start, start + timedelta(days=1))
    consumption = []
    for line in building_totalpower:
        consumption.append([line['datetime'],line['consumption']])

    total = pd.DataFrame(consumption, columns=['datetime', 'consumption'])
    total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
    total.set_index("datetime", inplace=True)
    total = total.resample('1h').mean()
    return total['consumption'].values.tolist()

In [10]:
from flask import jsonify
def overview():
    historic_overview = get_historic_overview()
    forecast = forecast_consumption()
    return {'historic': historic_overview, 'forecast': forecast}
overview()

2024-12-09 15:16:23


{'historic': [[1733842800.0, 273.0, 382.0, 32.76],
  [Timestamp('2024-12-10 00:00:00'),
   1586.1307371349096,
   0.0,
   153.12730180806676],
  [Timestamp('2024-12-09 23:00:00'), 1061.7041666666667, 0.0, 99.133375],
  [Timestamp('2024-12-09 22:00:00'),
   1576.32545201669,
   0.0,
   145.95025034770515],
  [Timestamp('2024-12-09 21:00:00'),
   1540.5944444444444,
   0.0,
   146.7137361111111],
  [Timestamp('2024-12-09 20:00:00'),
   1230.7621696801114,
   0.0,
   115.80082058414463],
  [Timestamp('2024-12-09 19:00:00'),
   834.6050069541029,
   0.0,
   76.54421418636996],
  [Timestamp('2024-12-09 18:00:00'),
   771.7680555555555,
   0.0,
   73.47481944444445],
  [Timestamp('2024-12-09 17:00:00'), 900.6314325452016, 0.0, 86.0006675938804],
  [Timestamp('2024-12-09 16:00:00'),
   2475.223611111111,
   0.2375,
   232.50169444444444]],
 'forecast': [1827.9958333333334,
  4149.086230876217,
  3368.0041666666666,
  3353.894297635605,
  2698.998611111111,
  2420.0111265646733,
  328.10416666